In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader

from deep_fib.sci_net import SCIBlockCfg, SCINet
from deep_fib.data import DeepFIBDataset
from deep_fib.core import step_function

from utils.data import Marconi100Dataset, get_train_test_split
from utils.training import training_loop
from utils.summary import SummaryWriter

c:\Users\zumo0\anaconda3\envs\masterAI\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 127] Impossibile trovare la procedura specificata
  warn(f"Failed to load image Python extension: {e}")


In [3]:
train, test = get_train_test_split(0.5, 42)

In [4]:
train = train[:10]
test = test[:4]

In [5]:
horizon = 1024
n_sample_per_data = 5

dataset_train = DeepFIBDataset(Marconi100Dataset(train, normalize=True), horizon, n_sample_per_data)
dataset_test = DeepFIBDataset(Marconi100Dataset(test, normalize=True), horizon, n_sample_per_data)

Loading: 100%|██████████| 4/4 [00:03<00:00,  1.12it/s]


In [6]:
data = dataset_train[0]
for k, d in data.items():
    print(k, d.size())

data torch.Size([1024, 460])
mask torch.Size([1024, 460])
label torch.Size([1024])


In [7]:
batch_size = 4
num_workers = 0

train_loader = DataLoader(dataset_train, batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(dataset_test, batch_size, shuffle=False, num_workers=num_workers)

In [8]:
for data in train_loader:
    for k, d in data.items():
        print(k, d.size())
    break

data torch.Size([4, 1024, 460])
mask torch.Size([4, 1024, 460])
label torch.Size([4, 1024])


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_encoder_levels=2
input_dim=460

model = SCINet(
    output_len=horizon,
    input_len=horizon,
    num_encoder_levels=num_encoder_levels,
    hidden_decoder_sizes=None,
    block_config=SCIBlockCfg(
        input_dim=input_dim,
        hidden_size=2,
        kernel_size=3,
        dropout=0.5,
    )
).float().to(device)

In [11]:
lr = 1e-3
step_size = 1

optim = Adam(model.parameters(), lr=lr)
lr_sched = StepLR(optim, step_size)

In [12]:
log_dir = "trash"

with SummaryWriter(log_dir) as writer:
    training_loop(
        model=model,
        step_function=step_function,
        num_epochs=3*step_size,
        train_dataloader=train_loader,
        test_dataloader=test_loader,
        device=device,
        optimizer=optim,
        lr_scheduler=lr_sched,
        writer=writer,
        save_path=log_dir + "/models"
    )

Epoch 0 - loss = 139.85572795252656 (111.17207134131235) - f1_score = 0.06739113950583661 (0.0345108811172505) - lr = 1.00e-03


Epoch 1 - loss = 89.62423183072003 (97.82855322866727) - f1_score = 0.06707509237655149 (0.0345108811172505) - lr = 1.00e-04


Epoch 2 - loss = 85.55501850497342 (97.02051833181669) - f1_score = 0.06737339936014672 (0.0345108811172505) - lr = 1.00e-05
